In [1]:
from osgeo import gdal,ogr
import pandas as pd
import geopandas
from shapely import wkt
import utm
import os
import fiona
import rasterio
import rasterio.mask

In [50]:
def ListofExtensionAndName(directory,extension):
     
        if len(directory) != None:
            import os
            FilesList = []
            FileName=[]
            for root, subdirectory, files in os.walk(directory):
                for file in files:
                    if file.endswith(extension):
                        FilesList.append(os.path.join(root,file))
                        base=os.path.basename(file)
                        FileName.append(os.path.splitext(base)[0])

            return FilesList,FileName
        else:
            print("no"+ extension +"file for this directory")

In [51]:
def ImageBoundry(FilePath):

        from osgeo import gdal,ogr
        ds = gdal.Open(FilePath)
        gt = ds.GetGeoTransform()  # captures origin and pixel size

        ULC = gdal.ApplyGeoTransform(gt, 0, 0) #Upper Left Corner
        URC = gdal.ApplyGeoTransform(gt, ds.RasterXSize, 0) #Upper Right Corner
        LLC = gdal.ApplyGeoTransform(gt, 0, ds.RasterYSize) #Lower Left Corner
        LRC = gdal.ApplyGeoTransform(gt, ds.RasterXSize, ds.RasterYSize) #Lower Right Corner
                   
        ring = ogr.Geometry(ogr.wkbLinearRing)
        ring.AddPoint_2D(ULC[0], ULC[1])
        ring.AddPoint_2D(URC[0], URC[1])
        ring.AddPoint_2D(LRC[0], LRC[1])
        ring.AddPoint_2D(LLC[0], LLC[1])
        ring.AddPoint_2D(ULC[0], ULC[1]) 
        poly=ogr.Geometry(ogr.wkbPolygon)
        poly.AddGeometry(ring)
        poly.ExportToJson
        return poly.ExportToJson()

In [174]:
def ImageBoundry_for_Gdal(FilePath):
        #for aspect data
        #imgname=FilePath

        from osgeo import gdal,ogr
        ds = gdal.Open(FilePath)
        gt = ds.GetGeoTransform()  # captures origin and pixel size

        ULC = gdal.ApplyGeoTransform(gt, 0, 0) #Upper Left Corner
        URC = gdal.ApplyGeoTransform(gt, ds.RasterXSize, 0) #Upper Right Corner
        LLC = gdal.ApplyGeoTransform(gt, 0, ds.RasterYSize) #Lower Left Corner
        LRC = gdal.ApplyGeoTransform(gt, ds.RasterXSize, ds.RasterYSize) #Lower Right Corner
                   
        ring = ogr.Geometry(ogr.wkbLinearRing)
        ring.AddPoint_2D(ULC[0], ULC[1])
        ring.AddPoint_2D(URC[0], URC[1])
        ring.AddPoint_2D(LRC[0], LRC[1])
        ring.AddPoint_2D(LLC[0], LLC[1])
        ring.AddPoint_2D(ULC[0], ULC[1]) 
        poly=ogr.Geometry(ogr.wkbPolygon)
        poly.AddGeometry(ring)
        poly.ExportToJson
        return ULC[0],ULC[1],LRC[0],LRC[1]

In [123]:
Targetimage=r"D:/Aspect_Result"  #mosaic aspect
SlopeSubset=r"D:/Aspect_result_subset"  #subset image path
RawImagePath=r"D://elevation_data" #raw dem data for getting extention

In [124]:
SubsetListName=ListofExtensionAndName(Targetimage,"tif")[1]


In [176]:
SubsetListName[293]

'EarthEnv-DEM90_N80W105'

In [ ]:
for i in SubsetListName[0:294]:
    rawimagepath=RawImagePath+"/"+i+".tif"
    targetimagepath=Targetimage+"/"+i+".tif"
    subsetimagepath=SlopeSubset+"/"+i+".tif"
    print("----Target Image-----")
    print(targetimagepath)
       
    boundry=ImageBoundry_for_Gdal(rawimagepath)
    
    print(boundry)
    print("-------------------")
    gdal.Translate(subsetimagepath,targetimagepath,projWin=[boundry[0],boundry[1],boundry[2],boundry[3]])

    

## Subset With Rasterio

In [ ]:
for i in SubsetListName:
    rawimagepath=RawImagePath+"\\"+i+".tif"
    targetimagepath=Targetimage+"\\"+i+".tif"
    subsetimagepath=SlopeSubset+"\\"+i+".tif"
    print(rawimagepath)
    imageboundry_subset=ImageBoundry(rawimagepath) #get the boudry
    gdf = geopandas.read_file(imageboundry_subset)
    geom_gdf=gdf.geometry

    print(geom_gdf.crs)
    #geom_gdf.to_file(SlopeSubset+i, driver='ESRI Shapefile', schema=None,)

    print(subsetimagepath)
    with rasterio.open(targetimagepath) as src:
        #open target image
        out_image, out_transform = rasterio.mask.mask(src, geom_gdf,
                                                      crop=True) #subset the image according to new extend
        
    #if we now the shape, we should define 
    #height= 6001
    #width=6001
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    with rasterio.open(subsetimagepath, "w", **out_meta) as dest:
        dest.write(out_image) #save the new subset image

    